Conectar drive personal

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Importar librerías necesarias e instalar modelado modelado de tópicos BERTopic

In [ ]:
import pandas as pd
!pip install bertopic

Importar modelo BERTopic e instanciarlo con parámetros de configuración básicos: lenguaje, cálculo de probabilidad de los tópicos obtenidos por documento y  seguimiento de las fases del modelo (verbose)

In [ ]:
from bertopic import BERTopic
topic_model = BERTopic(language="english", calculate_probabilities=False, verbose=True)

Primero se lee el dataset con los tweets limpios y pre-procesados. A continuación, se transforma la columna 'body' con los tweets a una lista de strings para utilizarla como entrada al modelo BERTopic. Finalmente, se obtienen los tópicos más relevantes del tópico

In [ ]:
#leer dataset
data_apple = pd.read_csv('/content/drive/MyDrive/TFG ADE/Data_Apple_Limpio.csv')
# transformar columna de tweets a lista de strings
data=(data_apple['body'].astype(str)).tolist()
# obtener los tópicos más relevantes del corpus
topics, _ = topic_model.fit_transform(data)

Se obtiene información general sobre los tópicos obtenidos por el modelo: nombre/identificador, términos representativos y documentos representativos (-1 es el tópico outlier, no se tiene en cuenta en el análisis)

In [ ]:
freq = topic_model.get_topic_info()
freq

,Topic,Count,Name,Representation,Representative_Docs
0,-1,33716,-1_iphone_phone_next_year,"[iphone, phone, next, year, good, tradedeal, d...","[closed took small loss want, stockfamily made..."
1,0,1957,0_research_patent_method_interface,"[research, patent, method, interface, electron...",[patent 10416800 device method graphical user ...
2,1,1349,1_robinhoodapp_sure_join_link,"[robinhoodapp, sure, join, link, free, stock, ...","[join robinhoodapp stock free sure link, join ..."
3,2,887,2_gspc_fall_surge_rise,"[gspc, fall, surge, rise, outlook, expectation...","[gspc nflx stock rise ahead open goldman, gspc..."
4,3,846,3_earnings_earningsseason_appstore_reporting,"[earnings, earningsseason, appstore, reporting...","[earnings, earnings, earnings]"
...,...,...,...,...,...
1553,1552,10,1552_peacock_comcast_sbgi_disca,"[peacock, comcast, sbgi, disca, viab, comcasts...","[comcast expects spend billion year peacock, c..."
1554,1553,10,1553_logi_realtime_subscribe_click,"[logi, realtime, subscribe, click, payment, 76...",[subscribe realtime option alert click link pa...
1555,1554,10,1554_behalf_korsinsky_klein_reminds,"[behalf, korsinsky, klein, reminds, class, sha...",[klein firm reminds investor class action beha...
1556,1555,10,1555_equity_175share_holdingstesla_holdingstes...,"[equity, 175share, holdingstesla, holdingstesl...","[upside 175share implication equity, equity al..."


Se extraen los n términos top del tópico más relevante/repetido (0) y la puntuación c-TF-IDF de cada uno

In [ ]:
#EXTRAER TOPICS
#topic mas frecuente
topic_model.get_topic(0)

[('research', 0.02325195318884244),
 ('patent', 0.022734258957638726),
 ('method', 0.015028652418491006),
 ('interface', 0.014220167404314223),
 ('electronic', 0.01360357726795762),
 ('device', 0.012226540403306826),
 ('tech', 0.011608928538690484),
 ('apparatus', 0.009768408037202084),
 ('graphical', 0.009379122881217618),
 ('user', 0.008205728349146284)]

Se visualiza la distancia intertópica del conjunto de tópicos iniciales, observando que los conjuntos se solapan entre ellos por lo que no estos no son lo suficientemente distintos y diferenciable

In [ ]:
#DISTANCIA INTERTOPICA INICIAL
topic_model.visualize_topics()

Se guarda el modelo para no perder la clasificación inicial cuando expire la sesión y se vuelve a cargar

In [ ]:
topic_model.save('/content/drive/MyDrive/TFG ADE/appletopicinicial')

In [ ]:
loaded_model = BERTopic.load('/content/drive/MyDrive/TFG ADE/appletopicinicial')

Se agrupan los tópicos para obtener las relaciones existentes entre ellos y se visualiza la jerarquía de los 25 más relevantes, con el objetivo de reducir el número de tópicos

In [ ]:
#JERARQUIA INICIAL QUE AYUDA A REDUCIR EL NUMERO DE TOPICOS
loaded_model.visualize_hierarchy(top_n_topics=25)

Se van reduciendo el número de tópicos y visualizando la distancia intertópica hasta obtener el número óptimo (15) en el que la distancia es lo suficientemente grande como para que estos sean diferenciables

In [ ]:
#PRUEBAS TOPIC REDUCTION Y DISTANCIA INTERTOPICA RESULTANTE
loaded_model.reduce_topics(data, nr_topics=15)
loaded_model.visualize_topics()

2023-11-11 10:46:21,030 - BERTopic - Reduced number of topics from 1558 to 15


De nuevo se obtiene la información general sobre los 15 tópicos, actualizando la representación de tópicos del modelo aumentando el rango de N-Gramas entre 1 y 3.

In [ ]:
loaded_model.get_topic_info()
loaded_model.update_topics(data, n_gram_range=(1,3))

Se guarda el modelo con los 15 tópicos resultantes para no perderlo cuando expire la sesión

In [ ]:
loaded_model.save('/content/drive/MyDrive/TFG ADE/appletopic15')

Se obtene la información sobre los documentos que sirvieron como base para la categorización de tópicos, incluyendo los documentos en sí, sus respectivos temas, el nombre de cada tema, las n palabras principales de cada tema, si es un documento representativo y la probabilidad de agrupación si se trata del modelo de agrupación

In [ ]:
loaded_model.get_document_info(data)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,patent portfolio company combined self driving...,-1,-1_stock_nflx_trade_market,"[stock, nflx, trade, market, iphone, nvda, int...",[investor heavily borrowing stock mean bull ma...,stock - nflx - trade - market - iphone - nvda ...,0.000000,False
1,different news tab open give,3,3_tab_different_open_give,"[tab, different, open, give, news, , , , , ]","[different news tab open give, different news ...",tab - different - open - give - news - - - ...,0.270597,True
2,billion cash hand billion fair amount aside ui...,-1,-1_stock_nflx_trade_market,"[stock, nflx, trade, market, iphone, nvda, int...",[investor heavily borrowing stock mean bull ma...,stock - nflx - trade - market - iphone - nvda ...,0.000000,False
3,using tariff battle illegal immigration realdo...,-1,-1_stock_nflx_trade_market,"[stock, nflx, trade, market, iphone, nvda, int...",[investor heavily borrowing stock mean bull ma...,stock - nflx - trade - market - iphone - nvda ...,0.000000,False
4,fang stock update leading marketwide charge lo...,0,0_stock_nflx_trade_option,"[stock, nflx, trade, option, nvda, baba, free,...",[stockfamily dump robinhood stockoptions optio...,stock - nflx - trade - option - nvda - baba - ...,1.000000,False
...,...,...,...,...,...,...,...,...
103858,imagine calling brokerdealer wanting good chun...,-1,-1_stock_nflx_trade_market,"[stock, nflx, trade, market, iphone, nvda, int...",[investor heavily borrowing stock mean bull ma...,stock - nflx - trade - market - iphone - nvda ...,0.000000,False
103859,yearly heck year 1999around buck lolhave frien...,-1,-1_stock_nflx_trade_market,"[stock, nflx, trade, market, iphone, nvda, int...",[investor heavily borrowing stock mean bull ma...,stock - nflx - trade - market - iphone - nvda ...,0.000000,False
103860,puuump last hour window dressing evil illumina...,0,0_stock_nflx_trade_option,"[stock, nflx, trade, option, nvda, baba, free,...",[stockfamily dump robinhood stockoptions optio...,stock - nflx - trade - option - nvda - baba - ...,0.783519,False
103861,dont discriminate,-1,-1_stock_nflx_trade_market,"[stock, nflx, trade, market, iphone, nvda, int...",[investor heavily borrowing stock mean bull ma...,stock - nflx - trade - market - iphone - nvda ...,0.000000,False


Se actualizan las etiquetas estándar de los tópicos resultantes asignadas por el modelo a unas etiquetas custom que faciliten la interpretabilidad de los mismos

In [ ]:
#custom labels
labels_apple={0: "Inversiones y comparativa entre empresas",
        1: "Patentes e investigación",
        2: "Directivos empresa",
        3: "Consultar noticias",
        4: "Mercado de cambios",
        5: "Análisis técnicos para estrategias de inversión",
        6:"Transacciones financieras",
              7:"Medios de comunicación y entretenimiento",
              8:"Tipos de contenido",
              9:"Cuestiones técnicas de producto",
              10:"California",
              11:"Indicadores",
              12:"Documentación y procedimientos",
              13:"Mapas"}
loaded_model.set_topic_labels(labels_apple)


Se representan en gráficos de barras los 15 tópicos ordenados por relevancia, en los cuales se obtienen los términos más repetidos por tópico y su puntuación c-TF-IDF

In [ ]:
modelo.visualize_barchart(top_n_topics=15,custom_labels=True, width=400)

Se representa la matriz de similitud entre los 10 primeros tópicos más relevantes, para comprobar si son lo suficientemente distintos (valores de similitud menor o igual a 0.5)

In [ ]:
loaded_model.visualize_heatmap(n_clusters=5, topics=[0,1,2,3,4,5,6,7,8,9],custom_labels=True, width=1200,height=1000)

Se añaden las columnas con la categorización por tópico (número y etiqueta custom) y las palabras clave de cada uno en el dataset

In [ ]:
data_apple['Tópico']=modelo.get_document_info(data_apple['body'])['Topic']
data_apple['Categoría']=modelo.get_document_info(data_apple['body'])['CustomName']
data_apple['Palabras clave']=modelo.get_document_info(data_apple['body'])['Top_n_words']

Se descartan los tweets que tratan sobre el tópico -1 (outlier)

In [ ]:
data_apple.drop(data_apple[(data_apple['Tópico'] ==-1)].index, inplace=True)

Se guarda el dataset con las nuevas columnas

In [ ]:
data_apple.to_csv('/content/drive/MyDrive/TFG ADE/AppleData15Topics.csv')

In [ ]:
data_apple.head()

Se obtiene la variación frecuencia por tópico a lo largo de la franja temporal de estudio

In [ ]:
#cambiar a mes
data_apple['post_date_only']= pd.to_datetime(data_apple['post_date']).dt.date
topics_over_time = modelo.topics_over_time(data_apple['body'].astype(str), data_apple['post_date_only'])

In [ ]:
modelo.visualize_topics_over_time(topics_over_time, topics=[0, 1, 2, 3, 4], custom_labels=True, width=1500)

In [ ]:
modelo.visualize_topics_over_time(topics_over_time, topics=[5, 6, 7, 8, 9], custom_labels=True, width=1500)

In [ ]:
modelo.visualize_topics_over_time(topics_over_time, topics=[10, 11, 12, 13, 13], custom_labels=True, width=1500)

Se exporta la evolución temporal por tópico para representarla en otra herramienta

In [ ]:
# evolucion temporal topicos
datos['post_date'] = pd.to_datetime(datos['post_date'])
topics = [0,1,2,3,4,5,6,7,8,9,10,11,12,13]

for topic in topics:
  topic_tweets = datos[datos['Tópico'] == topic]

  # Group the data by day and count the number of tweets per day
  t_per_day = topic_tweets.resample('D', on='post_date').count()
  columns_to_delete = ['body',  'Categoría', 'Palabras clave', 'Unnamed: 0', 'tweet_id']
  t_per_day = t_per_day.drop(columns=columns_to_delete)
  t_per_day = t_per_day.rename(columns={'Tópico': f'Tweets diarios sobre tópico {topic}'})
  t_per_day.to_csv(f'drive/MyDrive/TFG ADE/apple{topic}.csv')
